In [3]:
#Importing all the nessacary packages 
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.primitives import  BackendSampler
from qiskit.visualization import plot_distribution
from qiskit.circuit.library import MCMT, ZGate , XGate
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
import numpy as np
import math

In [4]:
def oracle(good_state):
    good_state = list(good_state) #turn the string input into a list
    n = len(good_state) #find the number of qubits for the circuit
    good_state.reverse() #reversing the list so that it matches qiskit indexing
    oracle = QuantumCircuit(n) #initialising the quantum circuit with n qubits

    for i in range(n): #adds an x gate for every qubit that should be zero for the good state
        if good_state[i] == '0': 
            oracle.x(i)
        
    oracle.compose(MCMT(ZGate(), n - 1, 1), inplace=True) #cz the whole circuit

    for i in range(n): #adds an x gate for every qubit that should be zero for the good state
        if good_state[i] == '0': 
            oracle.x(i)
    return oracle #returns the oracle as a quantum circuit

In [5]:
def Grover_op(good_state): 
    G = oracle(good_state) #renames the oracle
    n = len(list(good_state)) #find the number of qubits for the circuit
    list_of_qubits = np.arange(0,n,1).tolist() #creates a list of all qubits for indexing
    G.h(list_of_qubits) #hadamards the whole circuit
    G.x(list_of_qubits) #applys an x gate to all qubits
    G.compose(MCMT(ZGate(), n - 1, 1), inplace=True) #applys a controlled z gate to q_{n-1} as the target and the rest as controls
    G.x(list_of_qubits) #applys xgate to all qubits
    G.h(list_of_qubits) #hadamards the whole circuit
    return G #returns the grover operator as a circuit

In [16]:
def Grover(good_state): 
    G = oracle(good_state) #renames the oracle
    n = len(list(good_state)) #find the number of qubits for the circuit
    list_of_qubits = np.arange(0,n,1).tolist() #creates a list of all qubits for indexing
    qc = QuantumCircuit(n) #create a new quantum circuit qc
    G = Grover_op(good_state).to_gate(label='G') #turn the grover operator into a gate
    theta = math.asin(math.sqrt(1 / 2**n)) #calculate theta
    t = math.floor((math.pi / (4 * theta))) #calculate tau
    print(t)
    qc.h(list_of_qubits) #hadamard the whole circuit
    for i in range(t): #apply the grover operator tau times
        qc.append(G, list_of_qubits)
    qc.measure_all() #measure all the qubits
    return qc #return the grover circuit as a quantum circuit qc 

x = Grover('00010101')

12


In [9]:
good_state ='00010101' #define the good state 
service = QiskitRuntimeService() #choosing a service


#chosing a backend simulator 
backend = service.get_backend('ibmq_qasm_simulator') 

#setting up the sampler
sampler = BackendSampler(backend)

'''defining the job and runing it with 
10000 shots '''
job = sampler.run(Grover(good_state), shots = 10000) 

result = job.result() #save the results from the job

#retrieve the quasi probabilities
quasi_dist = result.quasi_dists[0] 

#plotting and saving the figure 
#plot_distribution(quasi_dist.binary_probabilities(), filename='../../Project/Images/Simulatedhist.png')

In [11]:
good_state ='00010101' #define the good state

'''setting optimization level and this 
was changed when nessacary'''
optimization_level = 3 #0,1,2,3

service = QiskitRuntimeService() #chosing a service
n = len(list(good_state)) #find the number of qubits for the circuit

#retreive the least busy backend
backend = service.least_busy(operational=True, min_num_qubits=n+10, simulator=False) 

#generate a pass manager
pm = generate_preset_pass_manager(optimization_level = optimization_level, backend=backend)

#set up the circuit to be deployed

Quantumcirc = Grover(good_state)
isa_circuit = pm.run(Quantumcirc)

#setting up sampler
sampler = Sampler(backend=backend) 

'''defining the job and runing it with 
10000 shots '''
job = sampler.run(isa_circuit, shots = 100) 

result = job.result() #retreive results
quasi_dist = result.quasi_dists[0] #get quasi probabilities 
time = job.usage_estimation #calculate the time taken
print(time) #output the time taken

#plotting and saving the figure 
#plot_distribution(quasi_dist.binary_probabilities(), filename=f'../../Project/Images/Qc{optimization_level}.png')

/Users/thelonious/miniconda3/envs/qiskit/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/thelonious/miniconda3/envs/qiskit/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/thelonious/miniconda3/envs/qiskit/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/thelonious/miniconda3/envs/qiskit/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/thelonious/miniconda3/envs/qiskit/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/thelonious

{'quantum_seconds': 251.08918400319723}


In [12]:
result = job.result() #retreive results
quasi_dist = result.quasi_dists[0] #get quasi probabilities 
time = job.usage_estimation #calculate the time taken
print(time) #output the time taken

#plotting and saving the figure 
plot_distribution(quasi_dist.binary_probabilities(), filename=f'../../Project/Images/Qc{optimization_level}.png')

{'quantum_seconds': 251.08918400319723}


In [14]:
quasi_dist = {
		"100": 0.029618730634513347,
		"101": 0.022039906177950878,
		"110": 0.03406680076483776,
		"111": 0.04174696911298392,
		"000": 0.03460986115983125,
		"001": 0.036152956469571576,
		"010": 0.03676516540733963,
		"011": 0.7649996102729716
	}
plot_distribution(quasi_dist, filename=f'../../Project/Images/Qc3.png')

In [138]:
#importing packages 
import time
import random 

'''defining the linear search
function that scans through each
state from a list and returns True once
the target value is found'''
def lsearch(states, good_state):
    for i in range(len(states)):
        if states[i] == good_state:
            return True

#defining the list of states
states  = ['111', '001', '010', '011', '100','101','110','111']

'''randomonly shuffle the list so 
it is an unstructued search'''
random.shuffle(states)

t0 = time.time()#record the time at start 
good_state  = '011' #define the good state

'''search for the good state 
from the list of states'''
lsearch(states,good_state)

t1 = time.time()#record the time after calculation
print(f'the state {good_state} was found') 
t = t1-t0 #calculate the total time
print(f'completed in {t} seconds') #time output

the state 011 was found
completed in 2.8848648071289062e-05 seconds
